## Overview

Kamodo provides a *functional* interface for space weather analysis, visualization, and knowledge discovery, allowing many problems in scientific data analysis to be posed in terms of function composition and evaluation. We'll walk through its general features here.

##  Kamodo objects

Users primarily interact with models and data through Kamodo objects. 

In [1]:
from kamodo.kamodo import Kamodo

### Function registration
Kamodo objects are essentially python dictionaries storing variable symbols as keys and their interpolating functions as values. New functions may be registered either at the initialization of the Kamodo object or later using dictionary bracket syntax.

In [2]:
kamodo = Kamodo('$x = t^2$')
kamodo['g'] = 'y-1'
kamodo

Kamodo([(x(t), <function _lambdifygenerated(t)>),
        (x, <function _lambdifygenerated(t)>),
        (g(y), <function _lambdifygenerated(y)>),
        (g, <function _lambdifygenerated(y)>)])

### Function composition
Kamodo automatically composes functions through specifying on the right-hand-side.

In [3]:
kamodo['f'] = 'g(x)'
kamodo

Kamodo([(x(t), <function _lambdifygenerated(t)>),
        (x, <function _lambdifygenerated(t)>),
        (g(y), <function _lambdifygenerated(y)>),
        (g, <function _lambdifygenerated(y)>),
        (f(t), <function _lambdifygenerated(t)>),
        (f, <function _lambdifygenerated(t)>)])

Here we have defined two functions $x(t)$, $g(y)$, and the composition $g∘f$. Kamodo was able to determine that $f$ is implicitly a function of $t$ even though we did not say so in $f$'s declaration.

#### Function evaluation
Kamodo uses sympy's ```lambdify``` function to turn the above equations into highly optimized functions for numerical evaluation. We may evaluate $f(t)$ for $t=3$ using "dot" notation:

In [4]:
kamodo.f(3)

8

where the return type is a numpy array. We could also have passed in a numpy array and the result shares the same shape:

In [5]:
import numpy as np
t = np.linspace(-5, 5, 100000)
result = kamodo.f(t)

In [6]:
assert(t.shape == result.shape)

### Unit conversion
Kamodo automatically handles unit conversions. Simply declare units on the left-hand-side of expressions using bracket notation.

In [35]:
kamodo = Kamodo('mass[kg] = x', 'vol[m^3] = y')

In [36]:
kamodo

Kamodo([(mass(x), <function _lambdifygenerated(x)>),
        (mass, <function _lambdifygenerated(x)>),
        (vol(y), <function _lambdifygenerated(y)>),
        (vol, <function _lambdifygenerated(y)>)])

Unless specified, Kamodo will assign the units for newly defined variables:

In [37]:
kamodo['rho'] = 'mass/vol'
kamodo

Kamodo([(mass(x), <function _lambdifygenerated(x)>),
        (mass, <function _lambdifygenerated(x)>),
        (vol(y), <function _lambdifygenerated(y)>),
        (vol, <function _lambdifygenerated(y)>),
        (rho(x, y), <function _lambdifygenerated(x, y)>),
        (rho, <function _lambdifygenerated(x, y)>)])

We may override the default behavior by simply naming the our chosen units in the left hand side.

In [38]:
kamodo['rho[g/cm^3]'] = 'mass/vol'
kamodo

Kamodo([(mass(x), <function _lambdifygenerated(x)>),
        (mass, <function _lambdifygenerated(x)>),
        (vol(y), <function _lambdifygenerated(y)>),
        (vol, <function _lambdifygenerated(y)>),
        (rho(x, y), <function _lambdifygenerated(x, y)>),
        (rho, <function _lambdifygenerated(x, y)>)])

!!! note
    Kamodo will raise an error if the left and right-hand-side units are incompatible.

Even though generated functions are unitless, the units are clearly displayed on the lhs. We think this is a good trade-off between performance and legibility.

We can verify that kamodo produces the correct output upon evaluation.

In [39]:
assert(kamodo.rho(3,8) == (3*1000.)/(8*100**3))

### Variable naming conventions
Kamodo allows for a wide array of variable names to suite your problem space, including greek, subscripts, superscripts.

In [40]:
kamodo = Kamodo(
    'rho = ALPHA+BETA+GAMMA',
    'rvec = t',
    'fprime = x',
    'xvec_i = xvec_iminus1 + 1',
    'F__gravity = G*M*m/R**2',
)
kamodo

Kamodo([(rho(ALPHA, BETA, GAMMA),
         <function _lambdifygenerated(ALPHA, BETA, GAMMA)>),
        (rho, <function _lambdifygenerated(ALPHA, BETA, GAMMA)>),
        (rvec(t), <function _lambdifygenerated(t)>),
        (rvec, <function _lambdifygenerated(t)>),
        (fprime(x), <function _lambdifygenerated(x)>),
        (fprime, <function _lambdifygenerated(x)>),
        (xvec_i(xvec_iminus1), <function _lambdifygenerated(xvec_iminus1)>),
        (xvec_i, <function _lambdifygenerated(xvec_iminus1)>),
        (F__gravity(G, M, R, m), <function _lambdifygenerated(G, M, R, m)>),
        (F__gravity, <function _lambdifygenerated(G, M, R, m)>)])

For more details on variable names, see the [Syntax](../Syntax/) section.

## Kamodofication

Many functions can not be written as simple mathematical expressions - they could represent simulation output or observational data. For this reason, we provide a ```@kamodofy``` decorator, which turns any callable function into a kamodo-compatible variable and adds metadata that enables unit conversion.

In [41]:
from kamodo.kamodo import kamodofy, Kamodo
import numpy as np

@kamodofy(units = 'kg/m^3', citation = 'Pembroke et. al, 2018')
def rho(x = np.array([3,4,5]), y = np.array([1,2,3])):
    """A function that computes density"""
    return x+y

kamodo = Kamodo(rho = rho)
kamodo['den[g/cm^3]'] = 'rho'
kamodo

Kamodo([(rho(x, y),
         <function __main__.rho(x=array([3, 4, 5]), y=array([1, 2, 3]))>),
        (rho, <function __main__.rho(x=array([3, 4, 5]), y=array([1, 2, 3]))>),
        (den(x, y), <function _lambdifygenerated(x, y)>),
        (den, <function _lambdifygenerated(x, y)>)])

In [42]:
kamodo.rho

<function __main__.rho(x=array([3, 4, 5]), y=array([1, 2, 3]))>

In [43]:
kamodo.den(3,4)

0.007

In [44]:
kamodo.rho.meta # PyHC standard

{'units': 'kg/m^3',
 'citation': 'Pembroke et. al, 2018',
 'equation': None,
 'hidden_args': []}

In [45]:
kamodo.rho.data # PyHC standard

array([4, 6, 8])

Original function doc strings and signatures passed through

In [46]:
help(kamodo.rho)

Help on function rho in module __main__:

rho(x=array([3, 4, 5]), y=array([1, 2, 3]))
    A function that computes density



In [47]:
kamodo.detail()

,lhs,rhs,symbol,units
"rho(x, y)",rho,None,"rho(x, y)",kg/m^3
"den(x, y)",den,"rho(x, y)/1000","den(x, y)",g/cm^3


# Visualization

Kamodo graphs are generated directly from function signatures by examining the structure of both output and input arguments. 

In [48]:
from plotting import plot_types
plot_types

plot_type  \
out_shape arg_shapes                                       
(1,)      ((N, M), (N, M), (N, M))         3d-parametric   
(N,)      ((N,),)                                1d-line   
          ((N,), (N,), (N,))              3d-line-scalar   
(N, 2)    ((N,),)                                2d-line   
          ((N, 2),)                            2d-vector   
(N, 3)    ((N,),)                                3d-line   
          ((N, 3),)                            3d-vector   
(N, M)    ((N,), (M,))                        2d-contour   
          ((N, M), (N, M))               2d-contour-skew   
          ((N, M), (N, M), (N, M))  3d-parametric-scalar   
          ((1,), (N, M), (N, M))                3d-plane   
          ((N, M), (1,), (N, M))                3d-plane   
          ((N, M), (N, M), (1,))                3d-plane   
(N, M, 1) ((1,), (N,), (M,))                    3d-plane   
          ((N,), (1,), (M,))                    3d-plane   
          ((N,), (M,), (1,))                    3d-plane   

                                                                  function  
out_shape arg_shapes                                                        
(1,)      ((N, M), (N, M), (N, M))       <function surface at 0x1306e0840>  
(N,)      ((N,),)                      <function line_plot at 0x11f6caa60>  
          ((N,), (N,), (N,))           <function line_plot at 0x11f6caa60>  
(N, 2)    ((N,),)                      <function line_plot at 0x11f6caa60>  
          ((N, 2),)                  <function vector_plot at 0x1306e0620>  
(N, 3)    ((N,),)                      <function line_plot at 0x11f6caa60>  
          ((N, 3),)                  <function vector_plot at 0x1306e0620>  
(N, M)    ((N,), (M,))              <function contour_plot at 0x1306e06a8>  
          ((N, M), (N, M))          <function contour_plot at 0x1306e06a8>  
          ((N, M), (N, M), (N, M))       <function surface at 0x1306e0840>  
          ((1,), (N, M), (N, M))           <function plane at 0x1306e07b8>  
          ((N, M), (1,), (N, M))           <function plane at 0x1306e07b8>  
          ((N, M), (N, M), (1,))           <function plane at 0x1306e07b8>  
(N, M, 1) ((1,), (N,), (M,))               <function plane at 0x1306e07b8>  
          ((N,), (1,), (M,))               <function plane at 0x1306e07b8>  
          ((N,), (M,), (1,))               <function plane at 0x1306e07b8>

Kamodo uses [plotly](https://plot.ly/python/) for visualization, enabling a rich array of interactive graphs and easy web deployment.

In [49]:
import plotly.io as pio
from plotly.offline import iplot,plot, init_notebook_mode
init_notebook_mode(connected=True)

In [50]:
@kamodofy(units = 'kg/m^3')
def rho(x = np.linspace(0,1, 20), y = np.linspace(-1, 1, 40)):
    """A function that computes density"""
    x_, y_ = np.meshgrid(x,y)
    return x_*y_

kamodo = Kamodo(rho = rho)
kamodo

Kamodo([(rho(x, y),
         <function __main__.rho(x=array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ]), y=array([-1.        , -0.94871795, -0.8974359 , -0.84615385, -0.79487179,
       -0.74358974, -0.69230769, -0.64102564, -0.58974359, -0.53846154,
       -0.48717949, -0.43589744, -0.38461538, -0.33333333, -0.28205128,
       -0.23076923, -0.17948718, -0.12820513, -0.07692308, -0.02564103,
        0.02564103,  0.07692308,  0.12820513,  0.17948718,  0.23076923,
        0.28205128,  0.33333333,  0.38461538,  0.43589744,  0.48717949,
        0.53846154,  0.58974359,  0.64102564,  0.69230769,  0.74358974,
        0.79487179,  0.84615385,  0.8974359 ,  0.94871795,  1.        ]))>),
        (rho,
         <function __main__.rho(x=array([0.        , 0.05263158, 0.10526316, 0.

We will generate an image of this function using plotly

In [51]:
fig = kamodo.plot('rho')
pio.write_image(fig, 'images/Kamodo_fig1.svg')

![fig1](images/Kamodo_fig1.svg)

See the [Visualization](../Visualization/) section for detailed examples.

## Latex I/O

Even though math is the language of physics, most scientific analysis software requires you to learn new programing languages. Kamodo allows users to write their mathematical expressions in LaTeX, a typesetting language most scientists already know:

In [52]:
kamodo = Kamodo('$rho[kg/m^3] = x^3$', '$v[cm/s] = y^2$')
kamodo['p[Pa]'] = '$\\rho v^2$'
kamodo

Kamodo([(rho(x), <function _lambdifygenerated(x)>),
        (rho, <function _lambdifygenerated(x)>),
        (v(y), <function _lambdifygenerated(y)>),
        (v, <function _lambdifygenerated(y)>),
        (p(x, y), <function _lambdifygenerated(x, y)>),
        (p, <function _lambdifygenerated(x, y)>)])

The resulting equation set may also be exported as a LaTeX string for use in publications:

In [53]:
print(kamodo.to_latex() + '\n.')

\begin{equation}\rho{\left(x \right)} [kg/m^3] = x^{3}\end{equation}\begin{equation}v{\left(y \right)} [cm/s] = y^{2}\end{equation}\begin{equation}p{\left(x,y \right)} [Pa] = \frac{\rho{\left(x \right)} v^{2}{\left(y \right)}}{10000}\end{equation}
.


# Simulation api

Kamodo offers a simple api for functions composed of each other.

Define variables as usual (order matters).

In [55]:
kamodo = Kamodo()
kamodo['y_iplus1'] = 'x_i + 1'
kamodo['x_iplus1'] = 'y_i - 2'
kamodo

Kamodo([(y_iplus1(x_i), <function _lambdifygenerated(x_i)>),
        (y_iplus1, <function _lambdifygenerated(x_i)>),
        (x_iplus1(y_i), <function _lambdifygenerated(y_i)>),
        (x_iplus1, <function _lambdifygenerated(y_i)>)])

Now add the ```update``` attribute to map functions onto arguments.

In [56]:
kamodo.x_iplus1.update = 'x_i'
kamodo.y_iplus1.update = 'y_i'

Create a simulation with initial conditions

In [57]:
simulation = kamodo.simulate(x_i = 0, steps = 5)
simulation #an iterator of arg, val dictionaries

<generator object simulate at 0x130985a20>

Run the simulation by iterating through the generator.

In [58]:
import pandas as pd
pd.DataFrame(simulation) # pandas conveniently iterates the results for display

,y_i,x_i
0,NaN,0
1,1.0,-1
2,0.0,-2
3,-1.0,-3
4,-2.0,-4
5,-3.0,-5
